In [24]:
# подключим диск
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [26]:
# импортируем необходимые библиотеки
import numpy as np
import pandas as pd
import nltk
from sklearn import feature_extraction

from nltk.tokenize import word_tokenize
# from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

In [ ]:
data = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/ЯП трансформации/q32.xlsx',
                     header=None, 
                     names=['rid', 'sid', 'theme_1', 'theme_2'],
                     skiprows=3, na_filter='')

In [ ]:
data.head()

,rid,sid,theme_1,theme_2
0,29819,35292,МЭШ,
1,29854,35039,,
2,29825,35292,Технологии,успех
3,29826,35100,,
4,29866,35044,Цифровая трансформация образовательной деятель...,


In [ ]:
data.shape[0]

14954

Приведем таблицу к удобному для работы виду, для этого нам нужно:

1. сконкатенировать колонки theme_1 и theme_2
2. очистить текст от спецсимволов и знаков препинания 

  (!) можно провести замену сокращений (например, ИП – индивидуальный         предприниматель), 

  (!) можно также удалить самые частые слова

3. перевести все слова в нижний регистр, провести стемминг/лемматизация
4. провести токинезацию
5. убрать строки с пустыми значениями
6. после провести векторизацию (bag of words)
7. не забыть про нормализацию
8. (!) можно так же попробовать создать вручную словарь с самыми частыми словами (МЭШ, успех, цифровизация, ...), затем сопоставить этим частым словам классы и назначить соответствующие классы соответствующим учительским оценкам, а остатки разбить на оставшееся кол-во классов

In [ ]:
# сконкатенируем theme_1 и theme_2
data['theme'] = data['theme_1'].astype(str) + ' ' + data['theme_2'].astype(str)
data.head()

,rid,sid,theme_1,theme_2,theme
0,29819,35292,МЭШ,,МЭШ
1,29854,35039,,,
2,29825,35292,Технологии,успех,Технологии успех
3,29826,35100,,,
4,29866,35044,Цифровая трансформация образовательной деятель...,,Цифровая трансформация образовательной деятель...


In [ ]:
data.shape[0]

14954

In [ ]:
# очистим текст от спецсимволов и знаков препинания
from string import punctuation

data['theme'] = data['theme'].apply(lambda x: x.translate(str.maketrans('', '', punctuation)))

In [ ]:
data.shape[0]

14954

In [ ]:
# проведем стемминг, при этом приведем все слова к нижнему регистру 
stemmer = SnowballStemmer('russian')

def MyTokenizer(themes):
    #ads = list(map(str.lower, ads)) # в случае списка строк, но здесь это не нужно
    
    themes_tokenized = [' '.join([stemmer.stem(w) for w in word_tokenize(theme.lower()) if w.isalpha()]) for theme in themes]
    return themes_tokenized

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
data['theme'] = MyTokenizer(data['theme'])

In [ ]:
data.shape[0]

14954

In [ ]:
# уберем строки с пустыми значениями 
# data = data[data['theme'] != ''] 

data['theme'].replace('', np.nan, inplace=True)
data.dropna(subset=['theme'], inplace=True)

In [ ]:
data.head()

,rid,sid,theme_1,theme_2,theme
0,29819,35292,МЭШ,,мэш
2,29825,35292,Технологии,успех,технолог успех
4,29866,35044,Цифровая трансформация образовательной деятель...,,цифров трансформац образовательн деятельн
5,29879,34977,сберкласс,,сберкласс
6,29888,34661,Информационные технологии в рамках ФГОС,,информацион технолог в рамк фгос


In [ ]:
data.shape[0]

9424

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer 

tfidf_vectorizer = TfidfVectorizer()
# bag of words
bow = tfidf_vectorizer.fit_transform(data['theme'])

In [ ]:
bow

<9424x2295 sparse matrix of type '<class 'numpy.float64'>'
	with 39476 stored elements in Compressed Sparse Row format>

In [ ]:
# scaler500_2 = MaxAbsScaler()
# bow500_2 = scaler500_2.fit_transform(bow500_2)

Мы привели отзывы учителей к виду, приемлемому для обучения моделей. Построим первую модель кластеризации. Будем использовать для этого метод k-средних:

In [ ]:
num_clusters = 10

# Метод k-средних - KMeans
from sklearn.cluster import KMeans

km = KMeans( n_clusters=num_clusters)
idx = km.fit(bow)

In [ ]:
clusters = km.labels_.tolist() 

# print(clusters) 
# print(km.labels_)

In [ ]:
len(clusters)

9424

In [ ]:
data['cluster'] = pd.DataFrame(clusters)

# кластеры от 1 до 10
data['cluster'] += 1
data['cluster'] = data['cluster'].apply(lambda x: str(x)[0:-2])

In [ ]:
data[~data['cluster'].astype(bool)].shape[0]

0

In [ ]:
data.head()

,rid,sid,theme_1,theme_2,theme,cluster
0,29819,35292,МЭШ,,мэш,9
2,29825,35292,Технологии,успех,технолог успех,7
4,29866,35044,Цифровая трансформация образовательной деятель...,,цифров трансформац образовательн деятельн,1
5,29879,34977,сберкласс,,сберкласс,5
6,29888,34661,Информационные технологии в рамках ФГОС,,информацион технолог в рамк фгос,4


In [ ]:
data.shape[0]

9424

In [ ]:
from sklearn.cluster import DBSCAN 

db = DBSCAN(eps=0.5, min_samples=70).fit(bow)
labels = db.labels_

In [ ]:
data['cluster_2'] = pd.DataFrame(labels)
data['cluster_2'] += 2
data['cluster_2'] = data['cluster_2'].apply(lambda x: str(x)[0:-2])

In [ ]:
data.head(50) 

,rid,sid,theme_1,theme_2,theme,cluster,cluster_2
0,29819,35292,МЭШ,,мэш,9,2
2,29825,35292,Технологии,успех,технолог успех,7,1
4,29866,35044,Цифровая трансформация образовательной деятель...,,цифров трансформац образовательн деятельн,1,1
5,29879,34977,сберкласс,,сберкласс,5,3
6,29888,34661,Информационные технологии в рамках ФГОС,,информацион технолог в рамк фгос,4,1
7,29867,34977,Цифровая грамотность,,цифров грамотн,6,1
8,29886,34977,Обучение Персонализированной системе образован...,,обучен персонализирова систем образован сберкласс,4,1
9,29953,35112,Методология и технология дистанционного обучен...,,методолог и технолог дистанцион обучен в образ...,4,1
11,29899,35292,"""Школа цифрового века""",,школ цифров век,10,1
12,29868,34977,Подготовка к итоговой аттестации.,Особенности преподавания математики.,подготовк к итогов аттестац особен преподаван ...,4,1


In [ ]:
from sklearn.cluster import AgglomerativeClustering 

aggclus = AgglomerativeClustering(n_clusters=num_clusters, affinity='euclidean') #affinity можно выбрать любое или попробовать все по очереди: cosine, l1, l2, manhattan
marks = aggclus.fit_predict(bow.toarray())

In [ ]:
data['cluster_3'] = pd.DataFrame(marks)
data['cluster_3'] += 1
data['cluster_3'] = data['cluster_3'].apply(lambda x: str(x)[0:-2])

In [ ]:
data.head() 

,rid,sid,theme_1,theme_2,theme,cluster,cluster_2,cluster_3
0,29819,35292,МЭШ,,мэш,9,2,5
2,29825,35292,Технологии,успех,технолог успех,7,1,1
4,29866,35044,Цифровая трансформация образовательной деятель...,,цифров трансформац образовательн деятельн,1,1,1
5,29879,34977,сберкласс,,сберкласс,5,3,3
6,29888,34661,Информационные технологии в рамках ФГОС,,информацион технолог в рамк фгос,4,1,1


In [ ]:
data.to_excel("results.xlsx")  

Попробуем другой подход.

Выдвинем следующую гипотезу: тезисы учителей о школах так или иначе соотносятся с экспертными оценками этих школ, т.е. не может быть ситуации, при которой учитель высказался плохо о школе, на примерно 2-3 из 10, в то время как эксперт дал оценку 8-9 из 10, с учетом его опыта и знания стратегии выставления оценок, другими словами, учитель может дать приблизительно верную оценку о школе на основе своего опыта работы в ней, эксперт же дает свою также корректную оценку на основе знания своего дела. Таким образом, мы можем сделать вывод о том, что оценки экспертов и учителей должны так или иначе откликаться и не отличаться сильно друг от друга.

Оираясь на данную гипотезу, попробуем использовать экспертные оценки (в числовом виде) школ как вектор правильных ответов для обучения модели.

In [ ]:
# создадим новый датафрейм, но уже с экспертными оценками школ
experts = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/ЯП трансформации/экспертные оценки.xlsx',
                     header=None, 
                     names=['rid', 'sid', 'mark_1', 'mark_2', 'overall_mark'],
                     skiprows=2)

# уберем строки со значениями 99
experts = experts[experts['overall_mark'] != 99] 

# оставим только нужные столбцы
experts = experts[['sid', 'overall_mark']]

In [ ]:
data.head(3)

,rid,sid,theme_1,theme_2,theme
0,29819,35292,МЭШ,,мэш
2,29825,35292,Технологии,успех,технолог успех
4,29866,35044,Цифровая трансформация образовательной деятель...,,цифров трансформац образовательн деятельн


In [ ]:
data = data.merge(experts, how='left', on='sid')

In [ ]:
data.head(3)

,rid,sid,theme_1,theme_2,theme,overall_mark
0,29819,35292,МЭШ,,мэш,NaN
1,29825,35292,Технологии,успех,технолог успех,NaN
2,29866,35044,Цифровая трансформация образовательной деятель...,,цифров трансформац образовательн деятельн,NaN


In [ ]:
# разобъем на обучающую (для которой есть ответы) и тестовую выборки
train_data = data[data['overall_mark'].isnull() == False]
test_data = data[data['overall_mark'].isnull() == True]

In [ ]:
train_data.shape[0] + test_data.shape[0]

9424

In [ ]:
train_data.head()

,rid,sid,theme_1,theme_2,theme,overall_mark
4,29888,34661,Информационные технологии в рамках ФГОС,,информацион технолог в рамк фгос,4.0
7,29953,35112,Методология и технология дистанционного обучен...,,методолог и технолог дистанцион обучен в образ...,4.0
17,30192,34775,Цифровые технологии в учебном процессе,,цифров технолог в учебн процесс,7.0
18,30991,35112,Внедрение целевой модели цифровой образователь...,,внедрен целев модел цифров образовательн сред ...,4.0
26,31510,35112,"Безопасное использование сайтов в сети ""Интерн...",,безопасн использован сайт в сет интернет в обр...,4.0


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

tfidf_vectorizer = TfidfVectorizer()
bow = tfidf_vectorizer.fit_transform(train_data['theme'])

In [ ]:
from sklearn.preprocessing import MaxAbsScaler

scaler = MaxAbsScaler()
bow = scaler.fit_transform(bow)

In [ ]:
from sklearn.linear_model import SGDClassifier 
from sklearn.metrics import accuracy_score 

# обучим SGDClassifier на полученной выборке
clf = SGDClassifier(max_iter=200, random_state=42)
clf.fit(bow, train_data['overall_mark'])

SGDClassifier(max_iter=200, random_state=42)

In [ ]:
test_bow = tfidf_vectorizer.transform(test_data['theme'])

In [ ]:
predictions = clf.predict(test_bow)

In [ ]:
test_data['overall_mark'] = predictions

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
test_data.head(25)

,rid,sid,theme_1,theme_2,theme,overall_mark
0,29819,35292,МЭШ,,мэш,7.0
1,29825,35292,Технологии,успех,технолог успех,8.0
2,29866,35044,Цифровая трансформация образовательной деятель...,,цифров трансформац образовательн деятельн,7.0
3,29879,34977,сберкласс,,сберкласс,6.0
5,29867,34977,Цифровая грамотность,,цифров грамотн,10.0
6,29886,34977,Обучение Персонализированной системе образован...,,обучен персонализирова систем образован сберкласс,6.0
8,29899,35292,"""Школа цифрового века""",,школ цифров век,9.0
9,29868,34977,Подготовка к итоговой аттестации.,Особенности преподавания математики.,подготовк к итогов аттестац особен преподаван ...,4.0
10,30360,35196,Работа с эжд,,работ с эжд,9.0
11,30196,35196,Современные технологии на уроках математики,,современ технолог на урок математик,6.0


In [ ]:
results_exp = pd.concat([train_data, test_data], ignore_index=True) 
results_exp['overall_mark'] = results_exp['overall_mark'].apply(lambda x: str(x)[0:-2])
results_exp['overall_mark'] = results_exp['overall_mark'].apply(lambda x: int(x))

In [ ]:
results_exp 

,rid,sid,theme_1,theme_2,theme,overall_mark
0,29888,34661,Информационные технологии в рамках ФГОС,,информацион технолог в рамк фгос,4
1,29953,35112,Методология и технология дистанционного обучен...,,методолог и технолог дистанцион обучен в образ...,4
2,30192,34775,Цифровые технологии в учебном процессе,,цифров технолог в учебн процесс,7
3,30991,35112,Внедрение целевой модели цифровой образователь...,,внедрен целев модел цифров образовательн сред ...,4
4,31510,35112,"Безопасное использование сайтов в сети ""Интерн...",,безопасн использован сайт в сет интернет в обр...,4
...,...,...,...,...,...,...
9419,76866,34722,использования цифровых технологий в условиях к...,,использован цифров технолог в услов,5
9420,60966,35271,Работа в МЭШ,,работ в мэш,7
9421,75493,34664,использование электронной доски на уроках,,использован электрон доск на урок,5
9422,89623,34976,курсы учителей английского языка с использован...,,курс учител английск язык с использован дистан...,8


In [ ]:
results_exp.to_excel("results_exp.xlsx") 

In [ ]:
test_data

,rid,sid,theme_1,theme_2,theme,overall_mark
0,29819,35292,МЭШ,,мэш,7.0
1,29825,35292,Технологии,успех,технолог успех,8.0
2,29866,35044,Цифровая трансформация образовательной деятель...,,цифров трансформац образовательн деятельн,7.0
3,29879,34977,сберкласс,,сберкласс,6.0
5,29867,34977,Цифровая грамотность,,цифров грамотн,10.0
...,...,...,...,...,...,...
9414,76866,34722,использования цифровых технологий в условиях к...,,использован цифров технолог в услов,5.0
9416,60966,35271,Работа в МЭШ,,работ в мэш,7.0
9417,75493,34664,использование электронной доски на уроках,,использован электрон доск на урок,5.0
9419,89623,34976,курсы учителей английского языка с использован...,,курс учител английск язык с использован дистан...,8.0


Попробуем перейти к 4-уровневой качественной шкале оценивания:
- 1-2 = 1
- 3-5 = 2
-	6-8 = 3
-	9-10 = 4

In [ ]:
experts.head()

,sid,overall_mark
0,34886,3
1,35038,2
2,35039,2
3,34887,3
4,34811,8


In [ ]:
# подгрузим изначальные файлы data и experts 

# experts.loc[:, 'overall_mark'][experts['overall_mark'] < 4] = 1
# experts.loc[:, 'overall_mark'][(experts['overall_mark'] >= 4) & (experts['overall_mark'] <= 5)]['overall_mark'] = 2
# experts.loc[:, 'overall_mark'][(experts['overall_mark'] >= 6) & (experts['overall_mark'] <= 7)]['overall_mark'] = 3
# experts.loc[:, 'overall_mark'][experts['overall_mark'] > 7]['overall_mark'] = 4

def scaling(x):
  if x < 3:
    x = 1
  elif (x >= 3) & (x <= 5):
    x = 2
  elif (x >= 6) & (x <= 8):
    x = 3
  else:
    x = 4
  return x

# переведем в 4-градационную шкалу
experts['overall_mark'] = experts['overall_mark'].apply(scaling)

experts.head()

,sid,overall_mark
0,34886,2
1,35038,1
2,35039,1
3,34887,2
4,34811,3


In [ ]:
experts['overall_mark'].value_counts()

3    65
2    53
1    17
4    15
Name: overall_mark, dtype: int64

In [ ]:
data = data.merge(experts, how='left', on='sid')

In [ ]:
# теперь проделаем все те же самые действия

# from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
# from sklearn.preprocessing import MaxAbsScaler
# from sklearn.linear_model import SGDClassifier
# from sklearn.metrics import accuracy_score 

train_data = data[data['overall_mark'].isnull() == False]
test_data = data[data['overall_mark'].isnull() == True]


tfidf_vectorizer = TfidfVectorizer()
bow = tfidf_vectorizer.fit_transform(train_data['theme'])


scaler = MaxAbsScaler()
bow = scaler.fit_transform(bow)


# обучим SGDClassifier на полученной выборке
clf = SGDClassifier(max_iter=200, random_state=42)
clf.fit(bow, train_data['overall_mark'])

test_bow = tfidf_vectorizer.transform(test_data['theme'])

predictions = clf.predict(test_bow)
test_data['overall_mark'] = predictions

results_exp_with_4_dim = pd.concat([train_data, test_data], ignore_index=True) 
results_exp_with_4_dim['overall_mark'] = results_exp_with_4_dim['overall_mark'].apply(lambda x: str(x)[0:-2])
results_exp_with_4_dim['overall_mark'] = results_exp_with_4_dim['overall_mark'].apply(lambda x: int(x))

results_exp_with_4_dim.to_excel("results_exp_with_4_dim.xlsx") 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
results_exp_with_4_dim

,rid,sid,theme_1,theme_2,theme,overall_mark
0,29888,34661,Информационные технологии в рамках ФГОС,,информацион технолог в рамк фгос,2
1,29953,35112,Методология и технология дистанционного обучен...,,методолог и технолог дистанцион обучен в образ...,2
2,30192,34775,Цифровые технологии в учебном процессе,,цифров технолог в учебн процесс,3
3,30991,35112,Внедрение целевой модели цифровой образователь...,,внедрен целев модел цифров образовательн сред ...,2
4,31510,35112,"Безопасное использование сайтов в сети ""Интерн...",,безопасн использован сайт в сет интернет в обр...,2
...,...,...,...,...,...,...
9419,76866,34722,использования цифровых технологий в условиях к...,,использован цифров технолог в услов,3
9420,60966,35271,Работа в МЭШ,,работ в мэш,3
9421,75493,34664,использование электронной доски на уроках,,использован электрон доск на урок,3
9422,89623,34976,курсы учителей английского языка с использован...,,курс учител английск язык с использован дистан...,3


Теперь попробуем убрать из обучающей выборки все московские школы, так как их большинство и они могут мешать обучению модели.

In [ ]:
# подгрузим изначальный файл experts, а также файл data_with_regions, где помимо привычных столбцов будет столбец с регионом школы 

# from string import punctuation

data_with_regions = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/ЯП трансформации/q32_with_regions.xlsx',
                     header=None, 
                     names=['rid', 'sid', 'reg', 'theme_1', 'theme_2'],
                     skiprows=3, na_filter='')

# сконкатенируем theme_1 и theme_2
data_with_regions['theme'] = data_with_regions['theme_1'].astype(str) + ' ' + data_with_regions['theme_2'].astype(str)

# очистим текст от спецсимволов и знаков препинания
data_with_regions['theme'] = data_with_regions['theme'].apply(lambda x: x.translate(str.maketrans('', '', punctuation)))

# проведем стемминг, при этом приведем все слова к нижнему регистру 
data_with_regions['theme'] = MyTokenizer(data_with_regions['theme'])

# уберем строки с пустыми значениями
data_with_regions['theme'].replace('', np.nan, inplace=True)
data_with_regions.dropna(subset=['theme'], inplace=True)

In [ ]:
data_with_regions.head()

,rid,sid,reg,theme_1,theme_2,theme
0,29819,35292,12,МЭШ,,мэш
2,29825,35292,12,Технологии,успех,технолог успех
4,29866,35044,45,Цифровая трансформация образовательной деятель...,,цифров трансформац образовательн деятельн
5,29879,34977,46,сберкласс,,сберкласс
6,29888,34661,22,Информационные технологии в рамках ФГОС,,информацион технолог в рамк фгос


In [ ]:
# проделаем те же действия, но теперь все школы московского региона переместим в тестовую выборку

data_with_regions = data_with_regions.merge(experts, how='left', on='sid')

# разобъем объекты на обучающую и тестовую выборки таким образом, чтобы все московские школы оказались в тестовой выборке
train_data = data_with_regions[(data_with_regions['overall_mark'].isnull() == False) & (data_with_regions['reg'] != 12)]
test_data = data_with_regions[(data_with_regions['overall_mark'].isnull() == True) |
                              ((data_with_regions['overall_mark'].isnull() == False) & (data_with_regions['reg'] == 12))]


tfidf_vectorizer = TfidfVectorizer()
bow = tfidf_vectorizer.fit_transform(train_data['theme'])


scaler = MaxAbsScaler()
bow = scaler.fit_transform(bow)


# обучим SGDClassifier на полученной выборке
clf = SGDClassifier(max_iter=200, random_state=42)
clf.fit(bow, train_data['overall_mark'])

test_bow = tfidf_vectorizer.transform(test_data['theme'])

predictions = clf.predict(test_bow)
test_data['overall_mark'] = predictions

results_exp_separately_moscow = pd.concat([train_data, test_data], ignore_index=True) 
results_exp_separately_moscow['overall_mark'] = results_exp_separately_moscow['overall_mark'].apply(lambda x: str(x)[0:-2])
results_exp_separately_moscow['overall_mark'] = results_exp_separately_moscow['overall_mark'].apply(lambda x: int(x))

results_exp_separately_moscow.to_excel("results_exp_separately_moscow.xlsx") 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Попробуем убрать из обучающей выборки не все московские школы, а только какую-то их часть.

In [ ]:
data_with_regions['reg'].value_counts()

12     4127
23      677
11      235
45      209
100     202
       ... 
433      11
72       11
57        7
62        7
48        4
Name: reg, Length: 84, dtype: int64

In [ ]:
# подгрузим изначальные файлы expertsи и data_with_regions и 
# проделаем те же действия, но теперь часть школ московского региона оставим в обучающей выборке, а
# часть - переместим в тестовую выборку

# from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
# from sklearn.preprocessing import MaxAbsScaler
# from sklearn.linear_model import SGDClassifier
# from sklearn.metrics import accuracy_score 

data_with_regions = data_with_regions.merge(experts, how='left', on='sid')

# возьмем рандомные 700 индексов строк с московскими школами
inxs = np.random.choice(np.array(data_with_regions[data_with_regions['reg'] == 12].index), 700)

train_data = data_with_regions[(data_with_regions['overall_mark'].isnull() == False) & 
                               ((data_with_regions['reg'] != 12) | (data_with_regions.index.isin(inxs)))]
test_data = data_with_regions[(data_with_regions['overall_mark'].isnull() == True) |
                              ((data_with_regions['overall_mark'].isnull() == False) & (data_with_regions['reg'] == 12))]


tfidf_vectorizer = TfidfVectorizer()
bow = tfidf_vectorizer.fit_transform(train_data['theme'])


scaler = MaxAbsScaler()
bow = scaler.fit_transform(bow)


# обучим SGDClassifier на полученной выборке
clf = SGDClassifier(max_iter=200, random_state=42)
clf.fit(bow, train_data['overall_mark'])

test_bow = tfidf_vectorizer.transform(test_data['theme'])

predictions = clf.predict(test_bow)
test_data['overall_mark'] = predictions

results_exp_700_moscow_in_train = pd.concat([train_data, test_data], ignore_index=True) 
results_exp_700_moscow_in_train['overall_mark'] = results_exp_700_moscow_in_train['overall_mark'].apply(lambda x: str(x)[0:-2])
results_exp_700_moscow_in_train['overall_mark'] = results_exp_700_moscow_in_train['overall_mark'].apply(lambda x: int(x))

results_exp_700_moscow_in_train.to_excel("results_exp_700_moscow_in_train.xlsx") 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
